In [13]:
import json
import numpy as np
import matplotlib.pyplot as plt

from trial_wo_classes import *

In [14]:
def Merge(dict1, dict2):
    # Check conflict before merging commands
    # for now dict1 is for pulses and dict2 is for sweeps
    key_delay_pairs_1 = [(key, value['delay']) for key, value in dict1.items()]
    key_delay_pairs_2 = [(key, value['delay']) for key, value in dict2.items()]
    # retrive 

    pprint(dict1)
    pprint(dict2)

    for pair1 in key_delay_pairs_1:
        for pair2 in key_delay_pairs_2:
            if abs(pair1[0]-pair2[0]) <  0.01:
                time = pair1[0]
                raise TypeError(f"-------Clash in command timings at {time}us -------- \n ")
    
    for pair1 in key_delay_pairs_1:
        for pair2 in key_delay_pairs_2:
            if pair1[0] < pair2[0] and pair1[0] > pair2[0]-pair2[1]:
                time = pair1[0]
                raise TypeError(f"-------Clash in command timings at {time}us -------- \n ")

    dict1.update(dict2)
    sorted_dict1 = dict(sorted(dict1.items()))
    print(sorted_dict1)
    print(dict1)
    return sorted_dict1

In [15]:
# def module_reset_sequence(command_dic, module_name):
    
#     previous_delay = 0
#     with open(modules_path+module_name+'.txt','w') as file:
#         file.write("### Commands for " + module_name+'\n')

#     with open(reset_path+'sweep_reset'+'.txt','w') as file:
#         file.write("### Reset commands for 3" + module_name+'\n')



#     for command_time in command_dic.keys():
#         delay = command_dic[command_time]["delay"]
#         command_type = list(command_dic[command_time].keys())
#         command_type.remove("delay")
#         command_on = "on"
#         command_off = "off"
#         command_sweep = "Sweep"
#         reset_text = ""

#         if command_type == [command_on]:
#             command_channels = command_dic[command_time][command_on]
#             tag = "Hello"
#             calc_delay = (command_time - previous_delay)*1000
#             command_text = f"\t\thvis.dio_send_trigger('Turn {command_on} triggers: {tag}', {module_name}, {command_channels}, {command_on}, {calc_delay})"
#             print(command_text)
#             previous_delay = command_time

#         if command_type == [command_off]:
#             command_channels = command_dic[command_time][command_off]
#             tag = "Hello"
#             calc_delay = (command_time - previous_delay)*1000
#             command_text = f"\t\thvis.dio_send_trigger('Turn {command_off} triggers: {tag}', {module_name}, {command_channels}, {command_off}, {calc_delay})"
#             print(command_text)
#             previous_delay = command_time
        
#         if command_type == [command_sweep]:
#             [Sweep_Reg_Number, Rising_Edge_Delay, Falling_Edge_Delay, Rising_Edge_Increment, Falling_Edge_Increment, command_channels] = command_dic[command_time][command_sweep]
#             tag = str(command_channels).replace('\'', '')
#             calc_delay = (command_time - previous_delay - delay)*1000
#             command_text = f"\t\thvis.dio_sweep('Sweeping {tag}', {module_name}, {command_channels}, {Sweep_Reg_Number}, {Rising_Edge_Increment}, {Falling_Edge_Increment}, {calc_delay})"             
#             print(command_text)
#             reset_text = f"\t\thvis.dio_sweep_reset('Sweeping {tag}', {module_name}, {command_channels}, {Sweep_Reg_Number}, {Rising_Edge_Increment}, {Falling_Edge_Increment}, {calc_delay})"             
#             # reset_text2 = f"\t\thvis.dio_sweep_reset('Sweeping)"             
#             with open(reset_path+'sweep_reset'+'.txt','a') as file:
#                 file.write(reset_text)
#             print("trying to reset")
#             previous_delay = command_time

#         with open(modules_path+module_name+'.txt', 'a') as file:
#             file.write(command_text+'\n')
        
        

In [16]:
def module_reset_sequence(command_dic, module_name):
    previous_delay = 0
    
    # Define file paths
    module_file_path = os.path.join(modules_path, f"{module_name}.txt")
    reset_file_path = os.path.join(reset_path, 'sweep_reset.txt')
    
    # Initialize the command and reset files
    with open(module_file_path, 'w') as module_file:
        module_file.write("### Commands for " + module_name + '\n')
    
    with open(reset_file_path, 'w') as reset_file:
        reset_file.write("### Reset commands for " + module_name + '\n')
    
    for command_time in sorted(command_dic.keys()):
        details = command_dic[command_time]
        delay = details.get("delay", 0)
        command_type_keys = [key for key in details.keys() if key != "delay"]
        
        if len(command_type_keys) != 1:
            print("Error: Command should have exactly one type ('on', 'off', 'Sweep')")
            continue
        
        command_type = command_type_keys[0]
        calc_delay = (command_time - previous_delay) * 1000
        
        if command_type == "on" or command_type == "off":
            command_channels = details[command_type]
            command_text = (
                f"\thvis.dio_send_trigger('Turn {command_type} triggers', "
                f"{module_name}, {command_channels}, '{command_type}', {calc_delay})"
            )
            previous_delay = command_time

        elif command_type == "Sweep":
            Sweep_Reg_Number, Rising_Edge_Delay, Falling_Edge_Delay, \
            Rising_Edge_Increment, Falling_Edge_Increment, command_channels = details[command_type]
            
            command_text = (
                f"\thvis.dio_sweep('Sweeping {command_channels}', {module_name}, {command_channels}, "
                f"{Sweep_Reg_Number}, {Rising_Edge_Increment}, {Falling_Edge_Increment}, {calc_delay})"
            )

            reset_text = (
                f"\thvis.dio_sweep_reset('Reset sweeping {command_channels}', {module_name}, {command_channels}, "
                f"{Sweep_Reg_Number}, {Rising_Edge_Delay}, {Falling_Edge_Delay})"
            )
            print(reset_text)
            # Write the reset command to the reset file
            with open(reset_file_path, 'a') as reset_file:
                reset_file.write("reset_text" + '\n')
            print("Hello_Ranjan")
            
            previous_delay = command_time

        # Write the command to the module file
        with open(module_file_path, 'a') as module_file:
            module_file.write(command_text + '\n')
        # with open(reset_file_path, 'a') as reset_file:
        #         reset_file.write("Hello" + '\n')


In [17]:
def print_sequence(on_off_pulses, module_name):
    keys = list(on_off_pulses.keys())
    
    with open(modules_path+module_name+'.txt','w') as file:
        file.write("### Commands for " + module_name+'\n')

    for i in range(len(keys)):
        if i == 0:
            if keys[i] == 0.0:
                print(module_name)
                print(
                    Fore.YELLOW,
                    "Adjusted Sequence to start at 0.01 us due to time constrains",
                    Fore.RESET,
                )
            for k, v in reversed(on_off_pulses[keys[i]].items()):
                if len(v) != 0:
                    tag = str(v).replace('\'', '')
                    command_text = f"\t\thvis.dio_send_trigger('Turn {k} triggers: {tag}', {module_name}, {v}, {k}, 10)"

                    with open(modules_path+module_name+'.txt', 'a') as file:
                        file.write(command_text+'\n')
        else:
            for k, v in reversed(on_off_pulses[keys[i]].items()):
                if len(v) != 0:
                    tag = str(v).replace('\'', '')
                    command_text = f"\t\thvis.dio_send_trigger('Turn {k} triggers {tag}', {module_name}, {v}, {k}, {calculate_difference(keys[i], keys[i-1])*1000})"

                    with open(modules_path+module_name+'.txt', 'a') as file:
                        file.write(command_text+'\n')

In [18]:
def script_gen(modules_path, templates_path, script_path, script_name):
    
    modules = os.listdir(modules_path)

    template_end_path = 'D:\MPQ\scripts\\templates\\template_end.txt'
    template_ini_path ='scripts\\templates\\template_ini_new.txt'
    sweep_reset_path = 'D:\MPQ\scripts\\reset\sweep_reset.txt'
    
    with open(template_ini_path, 'r') as file:
        template_ini = file.read()
    
    with open(template_end_path, 'r') as file:
        template_end = file.read()

    with open(sweep_reset_path, 'r') as file:
        sweep_reset = file.read()

    with open(script_path+script_name, 'w') as file:
        file.write(template_ini+'\n')
    
    with open(script_path+script_name, 'a') as file:
        file.write(sweep_reset+'\n')

    for module_sequence_path in modules:
        with open(modules_path + module_sequence_path, 'r') as file:
            module_sequence = file.read()
    
        with open(script_path+script_name, 'a') as file:
            file.write(module_sequence)
    
    with open(script_path+script_name, 'a') as file:
        file.write(template_end)
    
    print("python script generated \n")

In [19]:
# data = load_json("./sequences/test_json_arb.json")
data = load_json("./sequences/output.json")

delete_files_in_directory(modules_path)
delete_files_in_directory(reset_path)

for group in data["groups"]:

###########----------------For_Pulses----------------###########
    # Sorts all Pulses by the start time
    sorted_pulses_start = sorted(
        group["Pulses"].items(), key=lambda x: (x[1]["Start"], x[1]["Duration"])
    )
    print(group["name"])
    on_off_times = get_on_off(sorted_pulses_start)
    channel_list = get_all_channels(on_off_times)
    merged_on_off = merge_channels(on_off_times, channel_list, sorted_pulses_start)
    
    pulses_on_off = dict(sorted(merged_on_off.items(), key=lambda x: x))

###########----------------For_Sweeps----------------###########
    # Sorts sweeps by start time
    sorted_sweeps_start = sorted(
        group["Sweeps"].items(), key=lambda x: (x[1]["Start"])
    )
    sweeps_on = get_on_times(sorted_sweeps_start)

###########----------------Combine_Commands----------------###########
    final_command_dic = Merge(add_delay(pulses_on_off, "Pulses"), add_delay(sweeps_on, "Sweeps"))
    print("Merged the Dictionaries \n") 
    module_reset_sequence(final_command_dic, group["name"])

Directory cleared 

Directory cleared 

Module_DIO_2
{10.0: {'delay': 0.01, 'on': ['B0', 'B1']},
 1010.0: {'delay': 0.01, 'off': ['B0', 'B1']}}
{11: {'Sweep': [1, 0, 0.01, 0, 0.01, ['B2', 'B3']], 'delay': 0.11},
 100: {'Sweep': [2, 0, 0.01, 0, 0.01, ['B4']], 'delay': 0.08},
 200: {'Sweep': [3, 0, 0.01, 0, 0.01, ['B5']], 'delay': 0.08}}
{10.0: {'on': ['B0', 'B1'], 'delay': 0.01}, 11: {'Sweep': [1, 0, 0.01, 0, 0.01, ['B2', 'B3']], 'delay': 0.11}, 100: {'Sweep': [2, 0, 0.01, 0, 0.01, ['B4']], 'delay': 0.08}, 200: {'Sweep': [3, 0, 0.01, 0, 0.01, ['B5']], 'delay': 0.08}, 1010.0: {'off': ['B0', 'B1'], 'delay': 0.01}}
{10.0: {'on': ['B0', 'B1'], 'delay': 0.01}, 1010.0: {'off': ['B0', 'B1'], 'delay': 0.01}, 11: {'Sweep': [1, 0, 0.01, 0, 0.01, ['B2', 'B3']], 'delay': 0.11}, 100: {'Sweep': [2, 0, 0.01, 0, 0.01, ['B4']], 'delay': 0.08}, 200: {'Sweep': [3, 0, 0.01, 0, 0.01, ['B5']], 'delay': 0.08}}
Merged the Dictionaries 

	hvis.dio_sweep_reset('Reset sweeping ['B2', 'B3']', Module_DIO_2, ['B2', 

In [20]:
script_gen(modules_path, templates_path, script_path, "testttttt9.py")

python script generated 

